In [1]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sys
import itertools
from collections import namedtuple, defaultdict
from pathlib import Path
from enum import Enum
from os.path import splitext

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 1
%aimport analyze

with open('plots/style.json') as f:
    mpl.rcParams.update(json.load(f))

# Results

In [3]:
system = "perlmutter"
version = "v0.5.1"
oldversion = "v0.5.0"

## Current

In [4]:
new = analyze.Analysis(f"results/{system}/")
print(new)

Analysis for Celeritas v0.5.0-66+9f4ba76bd on perlmutter


In [5]:
analyze.calc_geo_frac(new)

count      mean       std
problem                     geo     arch                           
atlas-tilecal               orange  cpu     4.0  0.739838  0.004416
                            vecgeom cpu     4.0  0.646757  0.015900
cms-hgcal                   orange  cpu     4.0  0.729142  0.022510
                            vecgeom cpu     4.0  0.648874  0.006112
cms2018                     vecgeom cpu     4.0  0.694109  0.004425
cms2018+field+msc           vecgeom cpu     4.0  0.799035  0.012237
                                    gpu     4.0  0.975084  0.000936
testem15                    orange  cpu     4.0  0.452204  0.000668
testem15+field              orange  cpu     4.0  0.540102  0.002441
                                    gpu     4.0  0.580064  0.003742
testem15+field+msc          orange  cpu     4.0  0.631925  0.014896
                            vecgeom cpu     4.0  0.586340  0.008982
testem3-composite+field+msc orange  cpu     4.0  0.770957  0.021845
                                    gpu     4.0  0.866244  0.002580
                            vecgeom cpu     4.0  0.754893  0.015422
                                    gpu     4.0  0.899822  0.000332
testem3-composite+msc       orange  cpu     4.0  0.740531  0.009691
                            vecgeom cpu     4.0  0.707368  0.005524
testem3-expanded+field+msc  orange  cpu     4.0  0.773493  0.018145
                            vecgeom cpu     4.0  0.757802  0.008342
testem3-flat                orange  cpu     4.0  0.673786  0.008155
                            vecgeom cpu     4.0  0.564600  0.021943
testem3-flat+field          orange  cpu     4.0  0.711532  0.010168
testem3-flat+field+msc      orange  cpu     4.0  0.782479  0.022830
                                    gpu     4.0  0.867958  0.003304
                            vecgeom cpu     4.0  0.759557  0.010480
                                    gpu     4.0  0.931375  0.001303
testem3-flat+msc            orange  cpu     4.0  0.751436  0.006158

## Previous

In [6]:
old = analyze.Analysis(f"results-old/{system}/")
print(old, f"({oldversion})")

Analysis for Celeritas v0.5.0-dev.222+33193b7f2 on perlmutter (v0.5.0)


In [7]:
ftab = analyze.make_failure_table(old.failures())
ftab.to_frame()

,Failure


# Analysis

## Compare manually

In [19]:
def get_throughput(analysis):
    col = analysis.result['avg_event_per_time']
    return col.xs('vecgeom', level='geo').unstack('arch')

def get_slots(analysis):
    col = analysis.input['num_track_slots']
    return col.xs('vecgeom', level='geo').unstack('arch')

def compare(func):
    return 

In [27]:
rel_throughput = get_throughput(new) / get_throughput(old) - 1

In [47]:
rel_throughput.groupby('problem').mean()

arch,cpu,cpu+g4,gpu,gpu+g4,gpu+sync
problem,,,,,
atlas-tilecal,-0.402661,-0.001613,-0.047325,0.274179,NaN
cms-hgcal,-0.335999,0.018183,-0.026869,0.311110,NaN
cms2018,-0.416734,-0.065063,-0.106360,0.385430,NaN
cms2018+field+msc,-0.209769,-0.046486,-0.033685,2.037492,-0.033640
testem15+field,NaN,NaN,NaN,NaN,0.019817
testem15+field+msc,-0.298803,-0.006924,0.017690,0.190146,NaN
testem3-composite+field+msc,-0.129144,-0.017867,-0.001876,0.961296,-0.000729
testem3-composite+msc,-0.194831,-0.042603,-0.004617,0.352875,NaN
testem3-expanded+field+msc,-0.131094,-0.009980,-0.003637,0.946962,NaN


In [23]:
get_slots(new) / get_slots(old)

arch,cpu,cpu+g4,gpu,gpu+g4,gpu+sync
problem,,,,,
atlas-tilecal,16.0,1.0,1.0,4.0,NaN
cms-hgcal,16.0,1.0,1.0,4.0,NaN
cms2018+field+msc,16.0,1.0,1.0,4.0,1.0
cms2018,16.0,1.0,1.0,4.0,NaN
testem15+field+msc,16.0,1.0,1.0,4.0,NaN
testem15+field,NaN,NaN,NaN,NaN,1.0
testem3-composite+field+msc,16.0,1.0,1.0,4.0,1.0
testem3-composite+msc,16.0,1.0,1.0,4.0,NaN
testem3-expanded+field+msc,16.0,1.0,1.0,4.0,NaN


## Performance drop between versions

In [87]:
#analyses = {a.version: a for a in [frontier, frontier_v050]}
analyses = {
    oldversion: old,
    version: new,
}
throughput = {k: a.result['avg_event_per_time'] for k, a in analyses.items()}
plot_rel_to = old

In [88]:
def plot_ratio(result_dict):
    rel = analyze.summarize_instances(result_dict[version] / result_dict[oldversion])
    rel.dropna(inplace=True)
    fig, ax = plt.subplots(layout="constrained")
    plot_rel_to.plot_results(ax, rel)
    ax.grid(which='both')
    ax.legend()
    ax.text(0.98, 0.02,
        f"{version} / {oldversion} - 1 on {system}",
        va='bottom', ha='right',
        fontstyle='italic', color=(0.5,)*3, size='xx-small',
        transform=ax.transAxes,
        zorder=-100
    )

    return ax

In [89]:
steps = {k: a.result['avg_steps_per_primary'] for k, a in analyses.items()}

In [90]:
ax = plot_ratio(throughput)
ax.set_ylabel(f"Throughput [events/sec]")
ax.get_figure().savefig(f"results-old/{system}/rel-throughput.png", dpi=300)
ax.get_figure().savefig(f"results-old/{system}/rel-throughput.pdf", transparent=True)
plt.close()

In [91]:
ax = plot_ratio(steps)
ax.set_ylabel(f"Work [steps/primary]")
ax.get_figure().savefig(f"results-old/{system}/rel-work.png", dpi=300)
ax.get_figure().savefig(f"results-old/{system}/rel-work.pdf", transparent=True)
plt.close()